In [1]:
import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

# https://github.com/numba/numba/issues/4830 caching problem mixed with 
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/4
# maybe its possible to just call the func as func[1]


In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import numpy as np
from numba.types import int64, float_, ListType, void
from numba import typeof, typed, njit
from numba.typed import List
from numba.experimental import jitclass
from numba.core.types import FunctionType
import logging


# np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

# pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
@njit(cache=True)
def f1(x, y, callee):
    callee("adding x=" + str(x) + " and y=" + str(y))
    return x + y


@njit(cache=True)
def f2(x, y, callee):
    callee("multiplying x=" + str(x) + " and y=" + str(y))
    return x * y


@njit(cache=True)
def calc_test(x, y, text):
    return x * y


@njit(cache=True)
def print_test(x, y, text):
    print(text)
    return None


@njit(cache=True)
def callee_str(m):
    print(m)
    pass


@njit(cache=True)
def callee_pass(m):
    pass


@njit(cache=True)
def callee_logging(m):
    pass


call_str_type = types.void(types.unicode_type)
callee_func_type = types.FunctionType(call_str_type)
sig_str = types.int64(types.int64, types.int64, callee_func_type)

f_list = typed.List.empty_list(sig_str.as_type())
f_list.append(f1)
f_list.append(f2)

p_list = typed.List.empty_list(call_str_type.as_type())
p_list.append(callee_str)
p_list.append(callee_pass)
p_list.append(callee_logging)


@jitclass(
    [
        ("funcs", types.ListType(sig_str.as_type())),
        ("printers", types.ListType(call_str_type.as_type())),
    ]
)
class Handler:
    def __init__(self, funcs, printers):
        self.funcs = funcs
        self.printers = printers


@njit(cache=True)
def tester(f_list, p_list):
    x = Handler(f_list, p_list)

    for ind in range(10):
        time: types.int64 = 0
        hello: types.int64 = 0
        for dos in range(200):
            for bar in range(100):
                time += x.funcs[0](ind, dos, x.printers[1]) + bar
                hello += x.funcs[1](ind, 4, x.printers[1])
        result = time - hello
    return result

[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.f1-dac5d5d818.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.f1-dac5d5d818.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.f2-eb3e754af9.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.f2-eb3e754af9.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.callee_str-e507e98b18.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.callee_str-e507e98b18.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.callee_pass-325b917fa4.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.callee_pass-325b917fa4.py310.1.nbc'
[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3594183032.callee_logging-3be1055113.py310.nbi'
[cache] data loaded fro

In [4]:
@njit(cache=True)
def print_or_to_strr(number: float):
    print("candle body str")
    if number > 0:
        return "HitMaxTrades"
    # if number == 1:
    #     return "EntryFilled"
    # if number == 2:
    #     return "StopLossFilled"
    # elif number == 3:
    #     return "TakeProfitFilled"
    # elif number == 4:
    #     return "LiquidationFilled"
    # elif number == 5:
    #     return "MovedSLToBE"
    # elif number == 6:
    #     return "MovedTSL"


STR_FUNC_TYPE = types.unicode_type(types.float_)
STR_FUNC_LIST = typed.List.empty_list(STR_FUNC_TYPE.as_type())

STR_FUNC_LIST.append(print_or_to_strr)


@jitclass(
    [
        ("funcs", types.ListType(STR_FUNC_TYPE.as_type())),
        ("printers", types.ListType(call_str_type.as_type())),
    ]
)
class Handler:
    def __init__(self, funcs, printers):
        self.funcs = funcs
        self.printers = printers


@njit(cache=True)
def tester(f_list, p_list):
    x = Handler(f_list, p_list)

    for ind in range(10):
        time: types.int64 = 0
        hello: types.int64 = 0
        for dos in range(200):
            for bar in range(100):
                time += x.funcs[0](ind, dos, x.printers[1]) + bar
                hello += x.funcs[1](ind, 4, x.printers[1])
        result = time - hello
    return result

[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\1119597426.print_or_to_strr-80ee5d844f.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\1119597426.print_or_to_strr-80ee5d844f.py310.1.nbc'


ValueError: wrapper address of <class 'numba.core.registry.CPUDispatcher'> instance must be a positive integer but got 0 [sig=(float32,) -> unicode_type]

In [ ]:
tester(f_list=f_list, p_list=p_list)

In [ ]:
@njit(cache=True)
def helping(num):
    if num > 5:
        x = calc_test
        return x(1, 2, "hello")
    else:
        x = print_test
        x(1, 2, "hello")
    pass

In [ ]:
helping(5)
helping(20)

In [12]:
import os

from numba import njit
from numba.core.types import FunctionType, int64
from numba.extending import overload_method

# os.environ["NUMBA_DUMP_OPTIMIZED"] = "1"

func_type = FunctionType(int64(int64,))

@njit(int64(func_type), cache=True)
def f(g):
    return g(10)

@njit(cache=True)
def g(a):
    if a > 1:
        x= a + 1
    return x

print(f(g))

[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2053855360.f-308f8f616f.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2053855360.f-308f8f616f.py310.1.nbc'
[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2053855360.g-d89b7767ca.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2053855360.g-d89b7767ca.py310.1.nbc'
11
